# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Import Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json
import citipy

from scipy.stats import linregress
from citipy import citipy

# Import API key
from api_keys import weather_api_key
from api_keys import g_key

#Output data as CSV to appropriate file
output_csv = "output_data/cities.csv"

#create a range for latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
#first make lists to hold latitudes and longitudes. Also make a list to hold cities.
lats_lngs = []
cities = []

#generate a random series of latitude and longitude coordinates. Use numpy random function.
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)

#use the zip function to combine and place into the empty lats_lngs list
lats_lngs = zip(lats, lngs)

#use citipy to find the city closest to the random lat/lng combos.
for lat_lng in lats_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    #For each unique city, add it to the empty cities list
    if city not in cities:
        cities.append(city)

#print the cities to confirm sample size is adequate
print(cities)
len(cities)


['sorong', 'faanui', 'butaritari', 'karakendzha', 'bandarbeyla', 'scarborough', 'lisakovsk', 'cape town', 'punta arenas', 'puerto ayora', 'nikolskoye', 'kavaratti', 'olafsvik', 'hermanus', 'ilulissat', 'pochutla', 'rutland', 'busselton', 'east london', 'festus', 'nanakuli', 'severo-kurilsk', 'sao joao da barra', 'thompson', 'vaini', 'bethel', 'zeya', 'sao sepe', 'pastavy', 'ancud', 'arraial do cabo', 'saint anthony', 'rikitea', 'bonavista', 'tuktoyaktuk', 'kruisfontein', 'georgetown', 'ushuaia', 'bredasdorp', 'taveta', 'taolanaro', 'rocha', 'poum', 'belushya guba', 'ca mau', 'ribeira grande', 'albany', 'port alfred', 'jilmah', 'tual', 'karkaralinsk', 'tara', 'bathsheba', 'avarua', 'saskylakh', 'saldanha', 'cayenne', 'filingue', 'souillac', 'kavieng', 'oroville', 'terrak', 'ketchenery', 'oak harbor', 'springbok', 'attawapiskat', 'hamilton', 'bengkulu', 'illoqqortoormiut', 'cherskiy', 'luba', 'ponta do sol', 'yulara', 'castro', 'new norfolk', 'hobart', 'mys shmidta', 'rorvik', 'gurupa', 

574

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
#first create the necessary lists and counters to perform all the requested operations
city_list = []
cloudiness_list = []
country_list = []
date_list = []
humidity_list = []
lats_list = []
lngs_list = []
temp_max_list = []
windspeed_list = []
index_counter = 0
set_counter = 1

#perform API Calls
##point to the url needed for the data and set the units for the output to imperial
base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

#set the url for the query
query_url = f"{base_url}appid={weather_api_key}&units={units}&q="

#construct a for loop that matches city names with the city_list.
##Enumerate is the key here. Documentation: https://docs.python.org/3/library/functions.html?highlight=enumerate#enumerate
for index, city in enumerate(cities, start = 1):
    try:
        response = requests.get(query_url + city).json()
        #now insert all the names of the lists we made and append to them
        city_list.append(response["name"])
        cloudiness_list.append(response["clouds"]["all"])
        country_list.append(response["sys"]["country"])
        date_list.append(response["dt"])
        humidity_list.append(response["main"]["humidity"])
        lats_list.append(response["coord"]["lat"])
        lngs_list.append(response["coord"]["lon"])
        temp_max_list.append(response["main"]["temp_max"])
        windspeed_list.append(response["wind"]["speed"])
        
        #use if and else statements pointing to the counters to loop through
        if index_counter > 49:
            index_counter = 0
            set_counter = set_counter + 1
            
        else:
            index_counter = index_counter + 1
        
        #return the results with an f string
        print(f"Processing Data {index_counter} of Set {set_counter}:{city}")
        
    #handle the errors with an exception
    except(KeyError, IndexError):
        print("City not located. Moving on.")
print("I'm Finished.")

Processing Data 1 of Set 1:sorong
Processing Data 2 of Set 1:faanui
Processing Data 3 of Set 1:butaritari
City not located. Moving on.
Processing Data 4 of Set 1:bandarbeyla
Processing Data 5 of Set 1:scarborough
Processing Data 6 of Set 1:lisakovsk
Processing Data 7 of Set 1:cape town


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
#editing code for another push. had an error.
##create the dataframe with pd.DataFrame
weather_df = pd.DataFrame({
    "City" : city_list,
    "Country" : country_list,
    "Latitude" : lats_list,
    "Longitude" : lngs_list,
    "Maximum Temperature" : temp_max_list,
    "Humidity" : humidity_list,
    "Wind Speed" : windspeed_list,
    "Date" : date_list
})

#display the dataframe
weather_df.head()

###lists for easy copying and inclusion
#city_list = []
#cloudiness_list = []
#country_list = []
#date_list = []
#humidity_list = []
#lats_list = []
#lngs_list = []
#temp_max_list = []
#windspeed_list = []

In [ ]:
#count the data to make sure it's enough for the exercise
weather_df.count()

In [ ]:
#output the data results to the appropriate csv file for the hw
##documentation for .to_csv here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html
weather_df.to_csv('output_csv', index=False)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
weather_df.describe()

In [ ]:
#  Get the indices of cities that have humidity over 100%.
print('The Maximum Humidity observed in the DataFrame is 100%. No cities have humidity values greater than 100%. Skipping next step as instructed.')

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
#figure out one, and that's the template for all.
##use plt.scatter . Then label title, xlabel, ylabel. Gonna use the grid. Use .savefig for .png files.
###documentation: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html
####color documenation: https://matplotlib.org/stable/gallery/color/named_colors.html
plt.scatter(weather_df['Latitude'], weather_df['Maximum Temperature'], color="darkred", edgecolor="black")

plt.title("City Latitude versus Maximum Temperature (5/31/2021)")
plt.xlabel("Latitude")
plt.ylabel("Maximum Temperature (F)")
plt.savefig("../Images/Lat_v_MaxT.png")

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression